In [1]:
import sys,os
import uproot4 as uproot
import awkward1 as ak
import json
import numpy as np
import matplotlib.pyplot as plt
from fcc_python_tools.locations import loc
from fcc_python_tools import kinematics_flat
from particle import literals as lp
from fcc_python_tools import plotting
import tensorflow as tf
import zfit
import random
from tqdm import tqdm

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Roman']})
rc('text', usetex=True)

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "/cvmfs/sft-nightlies.cern.ch/lcg/views/dev3/Mon/x86_64-centos7-gcc8-opt/bin/python3"
  * The NumPy version is: "1.18.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: No module named 'numpy.core._multiarray_umath'


Particle mass definitions

In [ ]:
PDG_pi_m = lp.pi_plus.mass/1000.
PDG_K_m = lp.K_plus.mass/1000.

Functions to filter out charged pions and kaons

In [ ]:
def sel_pions(p):
    pi_cut = abs(p["mass"] - PDG_pi_m) < 1e-4
    pi = p[pi_cut]
    return pi

In [ ]:
def sel_kaons(p):
    k_cut = abs(p["mass"] - PDG_K_m) < 1e-4
    k = p[k_cut]
    return k

Function to make $D^0 \to K\pi$ combinations

In [ ]:
def make_D(pi, k):
    D = ak.cartesian({"k": k, "pi": pi})
    D_cut = np.sign(D["k","charge"]) != np.sign(D["pi","charge"])
    D = D[D_cut]

    D["mass"] = kinematics_flat.mass([D["k"], D["pi"]], [PDG_K_m, PDG_pi_m])
    
    PDG_D_m = lp.D_0.mass/1000.
    D_window = 0.05
    D_cut = abs(D["mass"] - PDG_D_m) < D_window
    D = D[D_cut]
    
    return D

Function to make $B^\pm \to D^0 \pi^\pm$ combinations

In [ ]:
def make_B(D, pi):
    B = ak.cartesian({"D": D, "pi": pi})
    B_cut = np.sign(B["D","k","charge"]) == np.sign(B["pi","charge"])
    B = B[B_cut]
    B["mass"] = kinematics_flat.mass([B["D","k"], B["D","pi"], B["pi"]], [PDG_K_m, PDG_pi_m, PDG_pi_m])
    return B

Loop over ROOT file in chunks of events (for memory management) and make combinations. At the end, combine all of the $B$ combinations from each chunk into one total array

In [ ]:
#Decay mode
mode = "Bu2D0Pi"

#File
path = "/eos/experiment/fcc/ee/tmp/flatntuples/Z_Zbb_Flavor_Uproot_test3/p8_ee_Zbb_ecm91.root"

#TTree in file
tree = "events"

#Container for the reco particles
p_c = 'RP'

#List of final B candidates
B_list = []

#Loop over chunks of events in the file for memory management
for events in tqdm(uproot.iterate(f"{path}:{tree}", how="zip", entrysteps="500 MB")):
    
    #Keep reco particles 
    p = events[p_c]
    
    #Apply momentum cut on particles
    p_cut = p["p"] > 1.
    p = p[p_cut]
    
    #Fetch the pions based on assigned track mass
    pi = sel_pions(p)
    
    #Fetch the kaons based on assigned track mass
    k = sel_kaons(p)
    
    #Create D candidates from pions and kaons 
    D = make_D(pi, k)
    
    #Make B candidates from D and pion
    B = make_B(D, pi)
    B_list.append(B)

B_tot = ak.concatenate(B_list) 